In [1]:
import pfb_functions as pfn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import csv

# define case names, the output figure frequency
figure_output_frequency = 1
# case_namelist = ['pfclm1_base1','pfclm1_base40','pfclm2_base1','pfclm2_base40']
case_namelist = ['pfclm2_base40']
layer_thicknesslist = [
[2,
1.8232,
0.9116,
0.9116,
0.6837,
0.2279,
0.0182,
0.0137,
0.0137,
0.0137,
0.0137,
0.0137,
0.0114,
0.0091,
0.0046,
0.0023,
]
]


In [2]:
# define output file path
sfpath = '/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/postprocessing/subsurfacestorage_cal'
outputfile_soilname_volume = 'pfclm_soil_storage_volume.csv'
outputfile_saproname_volume = 'pfclm_sapro_storage_volume.csv'

outputfile_soilname_depth = 'pfclm_soil_storage_depth.csv'
outputfile_saproname_depth = 'pfclm_sapro_storage_depth.csv'

num_case=len(case_namelist)

data_length=5 #168 for pfclm

soilwater_volume_matrix = np.zeros((data_length,num_case),dtype=float)
saprowater_volume_matrix = np.zeros((data_length,num_case),dtype=float)

soilwater_depth_matrix = np.zeros((data_length,num_case),dtype=float)
saprowater_depth_matrix = np.zeros((data_length,num_case),dtype=float)

ncols=198
nrows=198


In [3]:
def thickness_dom (layer_thickness,ncols,nrows,nlayers):
    thick_dom = np.zeros((ncols,nrows,nlayers),dtype=float)
    for i in range(nlayers):
#         thick_dom[:,:,i] = thick_dom[:,:,i] + layer_thickness[i]*5
        thick_dom[:,:,i] = layer_thickness[i]*5
    return thick_dom

In [4]:
def subsurface_storage (case_index,fpath,data_length,num_case,ncols,nrows,case_name,time_step,thick_dom, soilwater_volume_timeseries,saprowater_volume_timeseries,soilwater_depth_timeseries,saprowater_depth_timeseries):

#     print (time_step)
    fname_sat = '%s/%s.out.satur.%05d.pfb' %(fpath,case_name, time_step)
    fname_porosity = '%s/%s.out.porosity.pfb' %(fpath,case_name)
    
    satur = pfn.read(fname_sat)
    porosity = pfn.read(fname_porosity)
    
    depth_imcompressive = satur*porosity*thick_dom  #in unit of meter
    # get all soil layers from domain from soil bottom layer and above
    mask_ws_soil  =  depth_imcompressive[:,:,6:]
    mask_ws_soil_volume  =  mask_ws_soil*30*30
    # print (mask_ws_soil)
    mask_ws_sapro =  depth_imcompressive[:,:,1:6]
    mask_ws_sapro_volume =  mask_ws_sapro*30*30
    # print (mask_ws_sapro)
    # sum soil water storage for all cells
    total_soilwater_volume_map = np.sum(mask_ws_soil_volume, axis=2)
    total_saprowater_volume_map = np.sum(mask_ws_sapro_volume, axis=2)
#     

    
    total_soilwater_depth_map =np.sum(mask_ws_soil, axis=2)
    total_saprowater_depth_map =np.sum(mask_ws_sapro, axis=2)
#     print (total_soilwater_depth_map)
    # compute sum/mean for volume/depth and append the dataframe
    soilwater_volume_timeseries[time_step,case_index] =np.sum(total_soilwater_volume_map)
#     print (time_step,case_index,np.sum(total_soilwater_volume_map),soilwater_volume_timeseries)
    saprowater_volume_timeseries[time_step,case_index]=np.sum(total_saprowater_volume_map)
    
    soilwater_depth_timeseries[time_step,case_index] =total_soilwater_depth_map.mean()
#     print (time_step,case_index,total_soilwater_depth_map.mean(),soilwater_depth_timeseries)
    saprowater_depth_timeseries[time_step,case_index]=total_saprowater_depth_map.mean()
    
    return total_soilwater_volume_map,total_saprowater_volume_map,total_soilwater_depth_map,total_saprowater_depth_map,soilwater_volume_timeseries,saprowater_volume_timeseries,soilwater_depth_timeseries,saprowater_depth_timeseries

def plot_ws_2D (time_step,figure_output_frequency,soil_2D_data,sapro_2D_data,sfpath,case_name,variable):
    if time_step%figure_output_frequency==0:
        plt.figure(figsize=(10, 7))
        plt.pcolor(soil_2D_data,cmap='jet')
        plt.colorbar()
        plt.savefig('%s/fig/%s_soilwaterstorage_%s_%05d.png'% (sfpath,case_name,variable,time_step))
        plt.close()     
    
        plt.figure(figsize=(10, 7))
        plt.pcolor(sapro_2D_data,cmap='jet')
        plt.colorbar()
        plt.savefig('%s/fig/%s_saprowaterstorage_%s_%05d.png'% (sfpath,case_name,variable,time_step))
        plt.close()

def write_ws_1D (outputfile_soilname,outputfile_saproname, soilwater_data_timeseries,saprowater_data_timeseries):  

    np.savetxt(outputfile_soilname,soilwater_data_timeseries,delimiter=",")
    np.savetxt(outputfile_saproname,saprowater_data_timeseries,delimiter=",")


In [5]:
# Main function
def main(soilwater_volume_timeseries,saprowater_volume_timeseries,soilwater_depth_timeseries,saprowater_depth_timeseries):
    for case_index in range (num_case):
        case_name = case_namelist [case_index]
        print (case_name)
        layer_thickness = layer_thicknesslist [case_index]
        fpath = '/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_%s_s' % (case_name)
        
        for time_step in range (data_length):
            print (time_step)
            thick_dom=thickness_dom(layer_thickness,ncols,nrows,nlayers)
            (total_soilwater_volume_map,total_saprowater_volume_map,total_soilwater_depth_map,total_saprowater_depth_map, \
             soilwater_volume_timeseries,saprowater_volume_timeseries,soilwater_depth_timeseries,saprowater_depth_timeseries) \
            =subsurface_storage (case_index,fpath,data_length,num_case,ncols,nrows,case_name,time_step,thick_dom,soilwater_volume_timeseries,saprowater_volume_timeseries,soilwater_depth_timeseries,saprowater_depth_timeseries)
            # plot the storage depth, 2D
#             plot_ws_2D (time_step,figure_output_frequency,total_soilwater_depth_map,total_saprowater_depth_map,sfpath,case_name,'storageindepth')
#             plot_ws_2D (time_step,figure_output_frequency,total_soilwater_volume_map,total_saprowater_volume_map,sfpath,case_name,'storageinvolume')

        write_ws_1D (outputfile_soilname_depth,outputfile_saproname_depth,soilwater_depth_timeseries,saprowater_depth_timeseries)
        write_ws_1D (outputfile_soilname_volume,outputfile_saproname_volume,soilwater_volume_timeseries,saprowater_volume_timeseries)
main(soilwater_volume_matrix,saprowater_volume_matrix,soilwater_depth_matrix,saprowater_depth_matrix)

pfclm2_base40
0
1
2
3
4
